In [1]:
import cudf as pd
from sklearn.datasets import load_boston
boston = load_boston()
df = pd.DataFrame(data=boston.data, columns = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD',
        'TAX', 'PTRATIO', 'B', 'LSTAT'])
df['target'] = boston.target
df.head(5)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [2]:
import cupy as np
X=df.iloc[:,0:13]
Y=df.iloc[:,13]
Y=np.array(Y)
Y=Y.reshape(-1,1)

In [3]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X_scaled=scaler.fit_transform(X.as_matrix())

In [4]:
X_scaled

array([[-0.41978194,  0.28482986, -1.2879095 , ..., -1.45900038,
         0.44105193, -1.0755623 ],
       [-0.41733926, -0.48772236, -0.59338101, ..., -0.30309415,
         0.44105193, -0.49243937],
       [-0.41734159, -0.48772236, -0.59338101, ..., -0.30309415,
         0.39642699, -1.2087274 ],
       ...,
       [-0.41344658, -0.48772236,  0.11573841, ...,  1.17646583,
         0.44105193, -0.98304761],
       [-0.40776407, -0.48772236,  0.11573841, ...,  1.17646583,
         0.4032249 , -0.86530163],
       [-0.41500016, -0.48772236,  0.11573841, ...,  1.17646583,
         0.44105193, -0.66905833]])

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, Y, test_size=0.3, random_state=42)

In [6]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(354, 13)
(152, 13)
(354, 1)
(152, 1)


In [7]:
import cuml
from cuml import Ridge

In [8]:
method=['svd','eig']
msql=[]
mael=[]
r2l=[]

In [9]:
import cuml
for i in method:
    alpha = np.array([1e-5])
    ridge = Ridge(alpha = alpha, fit_intercept = True, normalize = False, solver = i)
    result_ridge = ridge.fit(X_train, y_train)
    print("Coefficients:")
    print(result_ridge.coef_)
    print("Intercept:")
    print(result_ridge.intercept_)
    preds = result_ridge.predict(X_test)
    msq=cuml.metrics.regression.mean_squared_error(y_test,preds)
    r2=cuml.metrics.regression.r2_score(y_test,preds)
    err=cuml.metrics.regression.mean_absolute_error(y_test,preds)
    msql.append(msq)
    mael.append(err)
    r2l.append(r2)
   

Coefficients:
[-1.14691398  0.83432582  0.33940664  0.79163618 -1.78472664  2.84783956
 -0.30429309 -2.9156248   2.11139957 -1.46519873 -1.96963459  1.07567768
 -3.90310711]
Intercept:
22.509454717075215
Coefficients:
[-1.14691398  0.83432582  0.33940664  0.79163618 -1.78472664  2.84783956
 -0.30429309 -2.9156248   2.11139957 -1.46519873 -1.96963459  1.07567768
 -3.90310711]
Intercept:
22.509454717075215


In [10]:
ridge_df=pd.DataFrame(method,columns=['Algorithm'])

In [11]:
ridge_df['mean square error']=msql
ridge_df['mean absolute error']=mael
ridge_df['r2 score']=r2l

In [12]:
ridge_df

,Algorithm,mean square error,mean absolute error,r2 score
0,svd,139.59932,8.983415,0.711226
1,eig,139.59932,8.983415,0.711226


In [13]:
from cuml.linear_model import Lasso

In [14]:
alpha=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]
msql=[]
mael=[]
r2l=[]

In [15]:
for i in alpha:
    ls = Lasso(alpha = i)
    result_lasso = ls.fit(X_train, y_train)
    print("Coefficients:")
    print(result_lasso.coef_)
    print("intercept:")
    print(result_lasso.intercept_)
    preds = result_lasso.predict(X_test)
    msq=cuml.metrics.regression.mean_squared_error(y_test,preds)
    r2=cuml.metrics.regression.r2_score(y_test,preds)
    err=cuml.metrics.regression.mean_absolute_error(y_test,preds)
    msql.append(msq)
    mael.append(err)
    r2l.append(r2)

    
    

Coefficients:
[-0.82672061  0.39323408  0.          0.80490657 -1.28995831  2.93306839
 -0.11769109 -2.12965495  0.51459017 -0.04896216 -1.8254361   0.95338885
 -3.93544882]
intercept:
22.513044683878363
Coefficients:
[-0.54652291  0.13003542  0.          0.76453477 -0.72479815  2.98159353
  0.         -1.40732985  0.          0.         -1.6527983   0.82881074
 -3.95660984]
intercept:
22.523952671727482
Coefficients:
[-0.46120429  0.          0.          0.69707703 -0.32009511  2.97395305
  0.         -0.86570225  0.          0.         -1.59411247  0.77083289
 -3.96966607]
intercept:
22.532051396235357
Coefficients:
[-0.38310237  0.          0.          0.63348026  0.          2.95238037
  0.         -0.4563307   0.          0.         -1.51542782  0.70882843
 -3.96107774]
intercept:
22.54307351782099
Coefficients:
[-0.2859226   0.          0.          0.5900941   0.          2.9194825
  0.         -0.24375816  0.          0.         -1.46579062  0.61515701
 -3.87002576]
intercept:
2

In [16]:
lass_df=pd.DataFrame(alpha,columns=['Alpha'])

In [17]:
lass_df['mean square error']=msql
lass_df['mean absolute error']=mael
lass_df['r2 score']=r2l
lass_df

,Alpha,mean square error,mean absolute error,r2 score
0,0.1,133.076405,8.773180,0.694318
1,0.2,129.072476,8.639153,0.680631
2,0.3,126.448094,8.547183,0.671090
3,0.4,124.306198,8.471644,0.662904
4,0.5,122.242198,8.403012,0.660372
5,0.6,120.311621,8.337624,0.656076
6,0.7,118.576353,8.273646,0.654003
7,0.8,116.923371,8.211475,0.651669
8,0.9,115.337842,8.151556,0.648476
9,1.0,113.893723,8.096987,0.644554
